## Hierarchical VAE

This is a simple implementation of a Hierarchical VAE.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import transforms
from datetime import datetime
import os
import matplotlib.pyplot as plt

## Download the dataset

In [2]:
def download_cifar10(data_path='./data'):
    """
    Download CIFAR-10 dataset and return trainset, testset, and classes

    Apply basic transformations to the data to normalize it between [-1, 1]
    """
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_path,
        train=True,
        download=True,
        transform=transform
    )

    testset = torchvision.datasets.CIFAR10(
        root=data_path,
        train=False,
        download=True,
        transform=transform
    )

    print(f"Training set size: {len(trainset)}")
    print(f"Test set size: {len(testset)}")
    
    # CIFAR-10 classes
    classes = ('plane', 'car', 'bird', 'cat', 'deer',
              'dog', 'frog', 'horse', 'ship', 'truck')
    
    return trainset, testset, classes

def get_dataloader(trainset, testset, batch_size=128):
    """Create DataLoader objects for training and testing"""
    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )
    
    test_loader = DataLoader(
        testset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )
    
    return train_loader, test_loader

## Define the Hierarchical VAE


In [3]:
class HierarchicalVAE(nn.Module):
    def __init__(self, latent_dims=[512, 256, 128]):
        super().__init__()
        
        self.latent_dims = latent_dims  # [z1: 512, z2: 256, z3: 128]
        
        # Encoder path
        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),    # 32x32 -> 16x16
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),   # 16x16 -> 8x8
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Flatten()  # 64 * 8 * 8 = 4096
        )
        
        # Latent variables means and logvars
        self.z1_mu = nn.Linear(64 * 8 * 8, latent_dims[0])
        self.z1_logvar = nn.Linear(64 * 8 * 8, latent_dims[0])
        
        self.encoder2 = nn.Sequential(
            nn.Linear(latent_dims[0], 512),  # z1: 512 -> hidden: 512
            nn.BatchNorm1d(512),
            nn.ReLU()
        )
        self.z2_mu = nn.Linear(512, latent_dims[1])
        self.z2_logvar = nn.Linear(512, latent_dims[1])
        
        self.encoder3 = nn.Sequential(
            nn.Linear(latent_dims[1], 256),  # z2: 256 -> hidden: 256
            nn.BatchNorm1d(256),
            nn.ReLU()
        )
        self.z3_mu = nn.Linear(256, latent_dims[2])
        self.z3_logvar = nn.Linear(256, latent_dims[2])
        
        # Decoder path
        self.decoder3 = nn.Sequential(
            nn.Linear(latent_dims[2], latent_dims[1]),  # z3: 128 -> hidden: 256
            nn.BatchNorm1d(latent_dims[1]),
            nn.ReLU()
        )

        # Add mixing layers for z2 and h3
        self.mix2 = nn.Sequential(
            nn.Linear(latent_dims[1] * 2, latent_dims[1]),  # (256 + 256) -> 256
            nn.BatchNorm1d(latent_dims[1]),
            nn.ReLU()
        )
        
        self.decoder2 = nn.Sequential(
            nn.Linear(latent_dims[1], latent_dims[0]),  # z2: 256 -> hidden: 512
            nn.BatchNorm1d(latent_dims[0]),
            nn.ReLU()
        )

        # Add mixing layers for z1 and h2
        self.mix1 = nn.Sequential(
            nn.Linear(latent_dims[0] * 2, latent_dims[0]),  # (512 + 512) -> 512
            nn.BatchNorm1d(latent_dims[0]),
            nn.ReLU()
        )

        self.decoder1_fc = nn.Sequential(
            nn.Linear(latent_dims[0], 64 * 8 * 8),  # z1: 512 -> hidden: 4096
            nn.BatchNorm1d(64 * 8 * 8),
            nn.ReLU()
        )
        self.decoder1_conv = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 8x8 -> 16x16
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),   # 16x16 -> 32x32
            nn.Tanh()
        )

    def reparameterize(self, mu, logvar):
        """Reparameterization trick."""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def encode(self, x):
        """Encodes the input into hierarchical latent variables."""
        # Level 1
        h1 = self.encoder1(x)
        z1_mu = self.z1_mu(h1)
        z1_logvar = self.z1_logvar(h1)
        z1 = self.reparameterize(z1_mu, z1_logvar)

        # Level 2
        h2_input = z1
        h2 = self.encoder2(h2_input)
        z2_mu = self.z2_mu(h2)
        z2_logvar = self.z2_logvar(h2)
        z2 = self.reparameterize(z2_mu, z2_logvar)
        
        # Level 3
        h3_input = z2
        h3 = self.encoder3(h3_input)
        z3_mu = self.z3_mu(h3)
        z3_logvar = self.z3_logvar(h3)
        z3 = self.reparameterize(z3_mu, z3_logvar)
        
        return [z3, z2, z1], [(z3_mu, z3_logvar), 
                             (z2_mu, z2_logvar), 
                             (z1_mu, z1_logvar)]

    def decode(self, zs):
        z3, z2, z1 = zs

        h3 = self.decoder3(z3)  # [batch_size, 256]

        combined_z2 = self.mix2(torch.cat([z2, h3], dim=1))
        h2 = self.decoder2(combined_z2)  # [batch_size, 512]

        combined_z1 = self.mix1(torch.cat([z1, h2], dim=1))
        h1_flat = self.decoder1_fc(combined_z1)  # [batch_size, 4096]
        
        # Reconstruct the image
        h1 = self.decoder1_conv(h1_flat.view(-1, 64, 8, 8))  # [batch_size, 3, 32, 32]

        return h1

    def forward(self, x):
        """Forward pass through the HVAE."""
        zs, mu_vars = self.encode(x)
        recon_x = self.decode(zs)
        return recon_x, mu_vars

    def random_samples(self, num_samples, device='cuda'):
        """Generates random samples from the HVAE using conditional sampling."""
        with torch.no_grad():
            # Sample z1 (lowest level) from prior
            z1 = torch.randn(num_samples, self.latent_dims[0]).to(device)
            
            # Generate z2 by encoding z1
            h2 = self.encoder2(z1)
            z2_mu = self.z2_mu(h2)
            z2_logvar = self.z2_logvar(h2)
            z2 = self.reparameterize(z2_mu, z2_logvar)
            
            # Generate z3 by encoding z2
            h3 = self.encoder3(z2)
            z3_mu = self.z3_mu(h3)
            z3_logvar = self.z3_logvar(h3)
            z3 = self.reparameterize(z3_mu, z3_logvar)
            
            # Decode all latent variables
            samples = self.decode([z3, z2, z1])
            samples = (samples + 1) / 2
            return samples.cpu()

In [4]:
def hvae_loss_function(recon_x, x, mu_vars, beta=1., epoch=None, warmup_epochs=50):
    # Reconstruction loss:
    # p(x|z) = N(x; μ(z), σ²I)
    # log p(x|z) = -0.5 * (log(2πσ²) + (x - μ(z))²/σ²)
    # log p(x|z) ∝ -0.5 * Σ(x - μ(z))²
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')

    # KL divergence loss
    # KL(N(μ,σ²) || N(0,1)) = 0.5 * (μ² + σ² - ln(σ²) - 1)
    kl_losses = []
    kl_weights = [1, .7, .5]
    for (mu, log_var), weight in zip(mu_vars, kl_weights):
        kl_i = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        kl_losses.append(kl_i * weight)
    total_kl_loss = torch.stack(kl_losses).mean()
 

    # Apply KL annealing if epoch is provided:
    # This is useful because usually the recon_loss
    # overwhelms the optimizer and we end up in a posterior collapse
    # where the KL Divergence never decreases
    # This is a simple way to gradually increase the KL loss
    # and prevent posterior collapse
    if epoch is not None:
        # Linearly increase beta from 0 to its final value
        beta_weight = min(epoch / warmup_epochs, 1.0) * beta
    else:
        beta_weight = beta

    total_loss = recon_loss + beta_weight*total_kl_loss
    
    return total_loss, recon_loss, total_kl_loss, kl_losses

def save_image_samples(model, data, writer, epoch, device):
    """Save original and reconstructed images to tensorboard"""
    model.eval()
    with torch.no_grad():
        # Get reconstructions
        data = data.to(device)
        recon_batch, _ = model(data)
        
        data_cpu = data[:8].cpu()
        recon_cpu = recon_batch.cpu()[:8]
        comparison = torch.cat([
            data_cpu,
            recon_cpu
        ])
        
        # Add images to tensorboard
        writer.add_images('Original_Reconstructed', comparison, epoch)

def train_epoch(model, train_loader, optimizer, device, writer, epoch):
    model.train()
    
    train_loss = 0
    train_recon_loss = 0
    # train_kl_losses = [0, 0, 0]  # For each level
    train_kl_loss = 0
    n_samples = len(train_loader.dataset)

    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        
        recon_batch, mu_vars = model(data)  # HVAE returns mu_vars list
        loss, recon_loss, total_kl, kl_losses = hvae_loss_function(
            recon_batch, data, mu_vars, epoch=epoch
        )
        
        loss.backward()
        train_loss += loss.item()
        train_recon_loss += recon_loss.item()
        train_kl_loss += total_kl.item()
        # for i, kl in enumerate(kl_losses):
        #     train_kl_losses[i] += kl.item()
            
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item() / len(data):.6f}')
    
    save_image_samples(model, data, writer, epoch, device)

    # Log metrics
    avg_loss = train_loss / n_samples
    avg_recon_loss = train_recon_loss / n_samples
    avg_kl_loss = train_kl_loss / n_samples
    
    writer.add_scalar('Loss/train/total', avg_loss, epoch)
    writer.add_scalar('Loss/train/reconstruction', avg_recon_loss, epoch)
    writer.add_scalar('Loss/train/kl_divergence', avg_kl_loss, epoch)
    # Log individual KL losses
    # for i, kl in enumerate(avg_kl_losses):
        # writer.add_scalar(f'Loss/train/kl_level_{i+1}', kl, epoch)
    
    return avg_loss

In [5]:

def train_vae(epochs=100, batch_size=128, learning_rate=1e-3, device="cuda"):
    # Get data
    trainset, testset, _ = download_cifar10()  # Using your existing function
    train_loader, test_loader = get_dataloader(trainset, testset, batch_size)
    
    # Initialize model, optimizer, and tensorboard
    model = HierarchicalVAE().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    log_dir = f'runs/HVAE_CIFAR10_{datetime.now().strftime("%Y%m%d-%H%M%S")}'
    writer = SummaryWriter(log_dir)
    
    # Training loop
    for epoch in range(1, epochs + 1):
        train_loss = train_epoch(model, train_loader, optimizer, device, writer, epoch)
        
        # Save a checkpoint every 10 epochs
        if epoch % 10 == 0:
            if not os.path.exists(f'{log_dir}/models'):
                os.makedirs(f'{log_dir}/models')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
            }, f'{log_dir}/models/hvae_checkpoint_epoch_{epoch}.pt')
    
    writer.close()

In [6]:
train_vae(epochs=200, batch_size=128, learning_rate=1e-3, device="cuda")


Files already downloaded and verified
Files already downloaded and verified
Training set size: 50000
Test set size: 10000
Train Epoch: 1 [0/50000 (0%)]	Loss: 1506.512207
Train Epoch: 1 [12800/50000 (26%)]	Loss: 213.966629
Train Epoch: 1 [25600/50000 (51%)]	Loss: 168.320374
Train Epoch: 1 [38400/50000 (77%)]	Loss: 146.051971
Train Epoch: 2 [0/50000 (0%)]	Loss: 136.676590
Train Epoch: 2 [12800/50000 (26%)]	Loss: 127.524582
Train Epoch: 2 [25600/50000 (51%)]	Loss: 118.169853
Train Epoch: 2 [38400/50000 (77%)]	Loss: 102.050323
Train Epoch: 3 [0/50000 (0%)]	Loss: 108.074844
Train Epoch: 3 [12800/50000 (26%)]	Loss: 99.339569
Train Epoch: 3 [25600/50000 (51%)]	Loss: 99.872574
Train Epoch: 3 [38400/50000 (77%)]	Loss: 95.560928
Train Epoch: 4 [0/50000 (0%)]	Loss: 97.865570
Train Epoch: 4 [12800/50000 (26%)]	Loss: 90.686798
Train Epoch: 4 [25600/50000 (51%)]	Loss: 99.730988
Train Epoch: 4 [38400/50000 (77%)]	Loss: 90.292816
Train Epoch: 5 [0/50000 (0%)]	Loss: 85.868813
Train Epoch: 5 [12800/5000

In [7]:
def generate_and_display_samples(log_dir, epoch=100, num_samples=64, latent_dims=[512, 256, 128], device='cuda'):
    """Generate and display random samples from the VAE decoder"""
    model = HierarchicalVAE(latent_dims=latent_dims).to(device)
    checkpoint = torch.load(f'{log_dir}/models/hvae_checkpoint_epoch_{str(epoch)}.pt', weights_only=True)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    with torch.no_grad():
        samples = model.random_samples(num_samples, device)
        
        # Create a grid of images
        fig, axes = plt.subplots(8, 8, figsize=(15, 15))
        for idx, ax in enumerate(axes.flat):
            # Convert from [C,H,W] to [H,W,C] format
            img = samples[idx].permute(1, 2, 0)
            
            ax.imshow(img)
            ax.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        return samples

In [8]:
# Initialize a new model instance
log_dir = './runs/HVAE_CIFAR10_20241227-204102'
samples = generate_and_display_samples(log_dir, 200)

# Optionally save to disk
torchvision.utils.save_image(samples, 'vae_samples.png', nrow=8, normalize=True)